In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [39]:
df = pd.read_csv('classification_cleaned.csv')
df.head()

,alpha,delta,cam_col,class,redshift,plate,MJD,photometric_filters
0,135.689107,32.494632,2,0,0.634794,5812,56354,21.422417
1,144.826101,31.274185,5,0,0.779136,10445,58158,22.918595
2,142.188790,35.582444,2,0,0.644195,4576,55592,21.999873
3,338.741038,-0.402828,3,0,0.932346,9149,58039,21.768664
4,345.282593,21.183866,3,0,0.116123,6121,56187,17.380674


In [40]:
# create X and y
X = df.drop('class', axis=1)
y = df['class']

In [41]:
df['class'].value_counts()

class
0    59445
2    21594
1    18961
Name: count, dtype: int64

In [42]:
from imblearn.under_sampling import RandomUnderSampler

# Using the under-sampling method to balance the values of the class column as you can see it previously. all those three values are not balanced
rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(X, y)


In [43]:
# after using the under-sampling method, lt's check the value counts of the 'class' column
pd.Series(y).value_counts()


class
0    18961
1    18961
2    18961
Name: count, dtype: int64

In [44]:
print("X_train shape:", X.shape)
print("y_train shape:", y.shape)

X_train shape: (56883, 7)
y_train shape: (56883,)


In [47]:
# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(45506, 7)
(45506,)
(11377, 7)
(11377,)


In [52]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test)

In [18]:
train = xgb.DMatrix(X_train, label=y_train)

In [55]:
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',
    'num_class': 10,
    'tree_method': 'gpu_hist'  # Enables GPU if available
}
bst = xgb.train(params, dtrain)


c:\Users\murta\OneDrive\Desktop\ML&DE\Third-semester\Deep_Learning\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [13:33:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [54]:
bst = xgb.train(params, dtrain)

c:\Users\murta\OneDrive\Desktop\ML&DE\Third-semester\Deep_Learning\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [13:32:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_gpus" } are not used.

  warnings.warn(smsg, UserWarning)
